In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
pip install accelerate -U

In [ ]:
import torch
torch.cuda.is_available()


False

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import re, sys
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install datasets transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('all-train.csv')
df = df.drop('Unnamed: 0', axis=1)
# select desired columns
df = df[['record_file', 'record_text', 'ABDOMINAL', 'CREATININE','MAJOR-DIABETES']]
df

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,162.xml,Record date: 2068-02-04\n\nASSOCIATED ARTHRITI...,met,not met,not met
1,176.xml,Record date: 2085-04-22\n\n \nThis patient wan...,met,not met,not met
2,189.xml,Record date: 2090-07-07\n\nWillow Gardens Care...,not met,met,met
3,214.xml,Record date: 2096-07-15\n\n\n\nResults01/31/20...,not met,not met,not met
4,200.xml,Record date: 2170-02-17\n\n \n\nReason for Vis...,met,not met,met
...,...,...,...,...,...
197,219.xml,Record date: 2070-01-05\n\n \n \n \n \n \nJanu...,not met,met,met
198,392.xml,"Record date: 2111-09-26\n\nCC: Ear pain, coug...",met,met,met
199,345.xml,Record date: 2072-11-25\n\n ...,not met,met,met
200,184.xml,Record date: 2062-01-29\n\nTRIBAL INTERNAL MED...,not met,met,met


In [ ]:
le = LabelEncoder()
label_cols = df.columns[2:]
label_cols
for col in label_cols:
  df[col] = le.fit_transform(df[col])

df.head()

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,162.xml,Record date: 2068-02-04\n\nASSOCIATED ARTHRITI...,0,1,1
1,176.xml,Record date: 2085-04-22\n\n \nThis patient wan...,0,1,1
2,189.xml,Record date: 2090-07-07\n\nWillow Gardens Care...,1,0,0
3,214.xml,Record date: 2096-07-15\n\n\n\nResults01/31/20...,1,1,1
4,200.xml,Record date: 2170-02-17\n\n \n\nReason for Vis...,0,1,0


In [ ]:

df.head()

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,162.xml,Record date: 2068-02-04\n\nASSOCIATED ARTHRITI...,0,1,1
1,176.xml,Record date: 2085-04-22\n\n \nThis patient wan...,0,1,1
2,189.xml,Record date: 2090-07-07\n\nWillow Gardens Care...,1,0,0
3,214.xml,Record date: 2096-07-15\n\n\n\nResults01/31/20...,1,1,1
4,200.xml,Record date: 2170-02-17\n\n \n\nReason for Vis...,0,1,0


In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=35)
test.reset_index(drop=True, inplace=True)
test.head(2)

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,333.xml,Record date: 2122-02-23\n\n\n**The source of t...,0,1,1
1,319.xml,Record date: 2085-08-21\n\n\nMIMA 5Z\n36 Santa...,1,1,0


In [ ]:
train, val = train_test_split(train, test_size=0.111, random_state=35)
val.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)
train.head(2)

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,362.xml,Record date: 2070-08-18\n\n ...,1,1,0
1,188.xml,Record date: 2066-03-05\n\n\n\n\n\nINTERNAL ME...,1,1,1


In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict
train_dataset =  Dataset.from_pandas(train[['record_text',	'ABDOMINAL',	'CREATININE',	'MAJOR-DIABETES']])



In [ ]:
test_dataset =  Dataset.from_pandas(test[['record_text',	'ABDOMINAL',	'CREATININE',	'MAJOR-DIABETES']])


In [ ]:
test_valid =  Dataset.from_pandas(val[['record_text',	'ABDOMINAL',	'CREATININE',	'MAJOR-DIABETES']])

In [ ]:
train_dataset

Dataset({
    features: ['record_text', 'ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES'],
    num_rows: 160
})

In [ ]:
test_valid

Dataset({
    features: ['record_text', 'ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES'],
    num_rows: 21
})

In [ ]:
%%capture
!pip install transformers datasets pandas torch

In [ ]:
import transformers
import datasets

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

Running on transformers v4.30.2 and datasets v2.13.1


## Imports

In [ ]:
import torch
from pathlib import Path
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

## Load data

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# path to train.csv test.csv and test_labels.csv
data_dir = Path("/content/gdrive/MyDrive/Colab Notebooks/data")
ds = (load_dataset("jigsaw_toxicity_pred", data_dir=data_dir, split='train')
        .train_test_split(train_size=800, test_size=200))
ds

Using custom data configuration default-2e028684d09fa340
Reusing dataset jigsaw_toxicity_pred (/root/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-2e028684d09fa340/1.1.0/b5a7e4444c940e3254416217128ad87ab7a53c9a54db4c72df349baecd5f43e6)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-2e028684d09fa340/1.1.0/b5a7e4444c940e3254416217128ad87ab7a53c9a54db4c72df349baecd5f43e6/cache-2bdf0adb9994a355.arrow and /root/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-2e028684d09fa340/1.1.0/b5a7e4444c940e3254416217128ad87ab7a53c9a54db4c72df349baecd5f43e6/cache-77e826447da52637.arrow


DatasetDict({
    train: Dataset({
        features: ['comment_text', 'identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic'],
        num_rows: 800
    })
    test: Dataset({
        features: ['comment_text', 'identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic'],
        num_rows: 200
    })
})

In [ ]:
ds=train_dataset
ds2=test_valid
ds3=test_dataset

## Preprocess data

In [ ]:
# create labels column
cols = ds.column_names
ds = ds.map(lambda x : {"labels": [x[c] for c in cols if c != "record_text"]})
ds

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Dataset({
    features: ['record_text', 'ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES', 'labels'],
    num_rows: 160
})

In [ ]:
# create labels column
cols = ds2.column_names
ds2 = ds2.map(lambda x : {"labels": [x[c] for c in cols if c != "record_text"]})
ds2

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Dataset({
    features: ['record_text', 'ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES', 'labels'],
    num_rows: 21
})

In [ ]:
ds2[0]

{'record_text': 'Record date: 2080-05-24\n\n54 yo f/u\n\n\n\nVS: 120/76  wt 390\n\n\n\n1. OBESITY - he feels his weight is down a little.  He has been trying to eat\n\nmainly meat and vegetables, but he is "stupid" sometimes and cheats.\n\n\n\n2. HYPERTENSION - in good range.  He has been making an effort to avoid Na.\n\n\n\n3. PROTEINURIA  - he has been off Naprosyn for about 6 weeks, has appt scheduled with Dr. Fry 6/05.  Will plan to repeat 24 hr urine for protein towards the end of next week.  His pedal edema has improved considerably, so possibly his albumin is better.\n\n\n\n4. DJD - Ultam is helpful, but not as good as Naprosyn.  Knees and ankles are stiff and painful, especially if he is active and then sits still for a while.\n\nAlso has some pain in left groin/thigh c/w tendonitis.\n\n\n\n\n\n\n\n\n\n--------------Medications----------\n\n\n\nFOLIC ACID     Tablet(s) PO  \n\nVIT C (ASCORBIC ACID)     Capsule(s) PO  \n\nVIT E (TOCOPHEROL-DL-ALPHA)     Capsule(s) PO  \n\nVIT B1

In [ ]:
# create labels column
cols = ds3.column_names
ds3 = ds3.map(lambda x : {"labels": [x[c] for c in cols if c != "record_text"]})
ds3

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Dataset({
    features: ['record_text', 'ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES', 'labels'],
    num_rows: 21
})

## Tokenize and encode

In [ ]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")

In [ ]:
def tokenize_and_encode(examples):
  return tokenizer(examples["record_text"], truncation=True)

In [ ]:
cols = ds.column_names
cols.remove("labels")
ds_enc = ds.map(tokenize_and_encode, batched=True, remove_columns=cols)
ds_enc

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 160
})

In [ ]:
cols = ds2.column_names
cols.remove("labels")
ds_enc2 = ds2.map(tokenize_and_encode, batched=True, remove_columns=cols)
ds_enc2

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 21
})

In [ ]:
cols = ds3.column_names
cols.remove("labels")
ds_enc3 = ds3.map(tokenize_and_encode, batched=True, remove_columns=cols)
ds_enc3

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 21
})

In [ ]:
# cast label IDs to floats
ds_enc.set_format("torch")
ds_enc = (ds_enc
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
# cast label IDs to floats
ds_enc2.set_format("torch")
ds_enc2 = (ds_enc2
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [ ]:
# cast label IDs to floats
ds_enc3.set_format("torch")
ds_enc3 = (ds_enc3
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

## Load model

In [ ]:
num_labels=3
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification").to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.we

## Load trainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:

args = TrainingArguments(
    "doc_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_enc,
    eval_dataset=ds_enc2,
    tokenizer=tokenizer,
)
trainer.train()

/content/doc_finetuned is already a clone of https://huggingface.co/thisishadis/doc_finetuned. Make sure you pull the latest changes with `repo.git_pull()`.
This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


Epoch,Training Loss,Validation Loss
1,No log,0.695092
2,No log,0.699963
3,No log,0.696297


Epoch,Training Loss,Validation Loss
1,No log,0.695092
2,No log,0.699963
3,No log,0.696297


TrainOutput(global_step=60, training_loss=0.6765115102132161, metrics={'train_runtime': 101.6401, 'train_samples_per_second': 4.723, 'train_steps_per_second': 0.59, 'total_flos': 63585485291520.0, 'train_loss': 0.6765115102132161, 'epoch': 3.0})

In [ ]:
from transformers import pipeline
model_checkpoint = "thisishadis/doc_finetuned"
classifier = pipeline(
    "text-classification", model=model_checkpoint, tokenizer=tokenizer
)



[{'label': 'LABEL_0', 'score': 0.5638298988342285}]

In [ ]:
classifier("the patient has pain in his head")

[{'label': 'LABEL_0', 'score': 0.5638298988342285}]

In [ ]:
classifier("she has high blood sugar")

[{'label': 'LABEL_1', 'score': 0.5417966246604919}]

In [ ]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
import shap
explainer = shap.Explainer(classifier)

'Record date: 2070-08-18\n\n                     KEKELA EMERGENCY DEPT VISIT\n \nDIETZ,SUSIE   538-41-21-0                    VISIT DATE: 08/18/70\nPRESENTING COMPLAINT:  Palpitations, mouth full of blood. \nHISTORY OF PRE'

In [ ]:
shap_values = explainer(['she has high blood sugar'])

In [ ]:
shap.plots.text(shap_values)

# اندازه گیری معیارها

In [ ]:
from torch.utils.data import DataLoader

tst_dataloader = DataLoader(
    ds_enc3, batch_size=10
)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, tst_dataloader = accelerator.prepare(
    model,  tst_dataloader
)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
model_ckpt = "thisishadis/doc_finetuned"

In [ ]:
num_labels=3
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification").to('cuda')

RuntimeError: ignored

In [ ]:
from tqdm.auto import tqdm
import torch

model.eval()
for batch in tst_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions=[]

        for o in list(outputs[1][0]):
          predictions.append([1 if oo>0.5 else 0 for oo in o])
        labels = batch["labels"]
        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        print(predictions_gathered,"******\n", labels_gathered)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)





NameError: ignored

In [ ]:
results = metric.compute()
print(
        "RESULTS ON TEST DATASET:\n",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        }
    )

RESULTS ON TEST DATASET:
 {'precision': 0.884375, 'recall': 0.8414271555996036, 'f1': 0.8623666835957339, 'accuracy': 0.981759913741464}


In [ ]:
from seqeval.metrics import performance_measure
print("confussion matrix (per token)")
performance_measure(true_labels,true_predictions)

confussion matrix (per token)


{'TP': 4225, 'FP': 243, 'FN': 366, 'TN': 28554}